# mysql과 python 연동
1. 라이브러리 설치
2. sql 서버의 정보를 등록
3. sql 서버에 접속
4. sql 쿼리문을 이용하여 데이터를 CRUD 작업

In [2]:
# pymysql (라이브러리) 설치

!pip install pymysql

In [3]:
import pymysql

In [4]:
# sql 서버의 정보를 입력

_db = pymysql.connect(
    user = 'root',
    password = '1234',
    host = 'localhost',
    db = 'ubion7',
    port = 3306
)

In [5]:
# cursor 변수 생성
# sql 쿼리문을 실행 시키기 위한 필수 변수
# Dictcursor는 sql 쿼리문이 실행되고 나온 데이터를 딕셔너리 형태로 받는다.
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [6]:
# """""" : 줄이 바뀌어도 이 사이는 다 str
sql = """
    select
    *
    from
    user_info
"""

cursor.execute(sql)

6

In [7]:
result = cursor.fetchall()

In [8]:
import pandas as pd

In [9]:
pd.DataFrame(result)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01011111111,2
2,test3,1111,test3,01099999999,2
3,test4,1234,test4,01011111111,1
4,test5,1234,test5,01011118888,1
5,test6,1234,test6,01011111111,2


In [10]:
sql = """
    select
    *
    from
    example    
"""

cursor.execute(sql)         # execute로 sql의 실행 
result = cursor.fetchall()  # fetchall로 sql을 받아온다
df = pd.DataFrame(result)
df

,Name,Gender,Phone
0,A,male,010-1234-5678
1,B,female,010-1234-5679
2,C,male,010-1234-5680
3,D,female,010-1234-5681
4,E,male,010-1234-5682
5,F,female,010-1234-5683
6,G,male,010-1234-5684
7,H,female,010-1234-5685
8,I,male,010-1234-5686
9,J,female,010-1234-5687


In [11]:
sql = """
    insert into
    user_info
    values
    ("test5", "1234", "test5", "01011118888", 1)
"""

cursor.execute(sql)

# 여기까지 하면 test5 실제 데이터베이스에 적용되지 않는다.
# execute 한 다음, commit 해야 한다.

IntegrityError: (1062, "Duplicate entry 'test5' for key 'user_info.PRIMARY'")

In [12]:
# commit
_db.commit()

# 여기까지 하고 MySQL에서 번개 누르거나, ctrl+enter 하면 변경 사항이 반영된다.

### sql 함수화
1. 첫 번째 함수
    - 매개변수 1개 (sql 쿼리문)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 변경하여 리턴
2. 두 번째 함수
    - 매개변수 1개 (sql 쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경
3. 세 번째 함수
    - 1번 함수와 2번 함수를 하나의 함수에서 실행이 될 수 있도록 생성

In [2]:
# 첫 번째 함수 (select 실행)
def sql_select(sql):
    
    # 쿼리문 실행
    cursor.execute(sql)  
    
    # 쿼리문에 대한 결과
    result = cursor.fetchall()

    df = pd.DataFrame(result)

    return df
    
    

In [3]:
sql = """
    select
    *
    from
    user_info    
"""

sql_select(sql)

NameError: name 'cursor' is not defined

In [4]:
# 두 번째 함수 (read, update, delete 모두 실행)
def _sql(sql, values) :
    
    cursor.execute(sql, values)
    _db.commit()
    

In [5]:
sql = """ 
    insert into
    user_info
    values
    (%s, %s, %s, %s, %s)

"""

values = ["test10", "12234", "test10", "01077777777", 1]

_sql(sql, values)

NameError: name 'cursor' is not defined

In [6]:
sql = """
    select
    *
    from
    user_info
"""

sql.find('select')
sql.replace

<function str.replace(old, new, count=-1, /)>

In [7]:
sql = """ 
    insert
    *
    from
    user_info
"""

sql.find('select')

-1

In [8]:
def sql_query(sql, values = []) :
    # if (sql.find('select') != -1) and (sql.find('select') < 10) :
    if (sql.replace('\n', '').strip().startswith('select')) :
        cursor.execute(sql, values)
        result = cursor.fetchall()
        result = pd.DataFrame(result)

    else :
        cursor.execute(sql, values)
        _db.commit()
        result = "Query OK"
    return result

In [ ]:
sql = """
    select
    *
    from
    user_info
"""

sql_query(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test10,12234,test10,01077777777,1
2,test2,1234,test2,01011111111,2
3,test3,1111,test3,01099999999,2
4,test4,1234,test4,01011111111,1
5,test5,1234,test5,01011118888,1
6,test6,1234,test6,01011111111,2


In [9]:
sql = """
    delete
    from
    user_info
    where
    user_id = %s
"""
    # values값을 여러개로 넣고 싶으면
    # user_id in (%s,.. 갯수만큼)

values = ['test10']

sql_query(sql, values)

NameError: name 'cursor' is not defined

In [69]:
import mod_sql
import imp
imp.reload(mod_sql)

<module 'mod_sql' from 'c:\\Users\\shoni\\Documents\\GitHub\\test\\shoni_python\\230417\\mod_sql\\__init__.py'>

In [72]:
test = mod_sql.Database(
        _host = 'localhost',
        _user = 'root',
        _pass = '1234',
        _db = 'ubion7',
        _port = 3306
)

In [19]:
sql = """
    select
    *
    from
    user_info
"""

test.sql_query(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01011111111,2
2,test3,1111,test3,01099999999,2
3,test4,1234,test4,01011111111,1
4,test5,1234,test5,01011118888,1
5,test6,1234,test6,01011111111,2


In [43]:
test2 = mod_sql.Database(
    '172.16.11.148',
    'ubion',
    '1234',
    'ubion7',
    3306
)

In [22]:
sql = """
    select
    *
    from
    user_info
"""

test2.sql_query(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,0101111111,2
2,test4,1234,test4,01043432432,1
3,test5,1234,test5,01011118888,1
4,test6,1234,test6,01011111111,2
5,test7,1234,test7,01030423423,2


In [44]:
sql = """ 
    insert into
    user_info
    values
    ("test1111", "1234", "test", "01022222222", 1)
"""

test2.sql_query(sql)

'Query OK'

In [45]:
sql = """
    select
    *
    from
    user_info
"""

test2.sql_query(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,MR.no,9999,bbbccc,01012341234,1
1,Sexyyeongsang,9999,tsast,123213123,1
2,teddd3,12343d5,tesdt12,0107377577d7,1
3,test10,1223,test,01033338888,1
4,test1111,1234,test,01022222222,1
5,test11111,1234,test,01011111111,1
6,test99,1465,test99,01077777777,1


In [73]:
sql = """
    select
    `Name` as '이름',
    `Phone` as '휴대폰번호'
    from
    example
    where
    Gender = %s
    limit 3
"""
# column 에는 ``로 묶어주는 것이 좋다. 공백이 있을 경우 따로 보기 때문에. 따옴표 아니구 1 옆에 있는 거임~
# limit는 몇개만 뽑을지 정하는 것. .head(3) 과 동일
values = ['female']

test.sql_query(sql, values)

,이름,휴대폰번호
0,B,010-1234-5679
1,D,010-1234-5681
2,F,010-1234-5683


In [42]:
sql = """
    select
    createDt as '등록일시',
    deathCnt as '사망자',
    decideCnt as '감염자'
    from
    corona
"""
test.sql_query(sql)

,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


In [56]:
test.sql_close()

### 같은 작업을 pandas로 해보자

In [47]:
import pandas as pd

In [64]:
data = pd.read_csv("../csv/corona.csv")

In [65]:
data.head(2)

,Unnamed: 0,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,0,2022-06-08 09:09:05.982,24305,18188200,904,20220608,00:00,NaN,NaN,NaN
1,1,2022-06-07 09:09:00.897,24299,18174842,903,20220607,00:00,2022-06-08 09:10:36.846,NaN,NaN


In [62]:
data.columns

Index(['Unnamed: 0', 'createDt', 'deathCnt', 'decideCnt', 'seq', 'stateDt',
       'stateTime', 'updateDt', 'accExamCnt', 'accDefRate'],
      dtype='object')

In [66]:
data.drop(['Unnamed: 0', 'seq', 'stateDt',
       'stateTime', 'updateDt', 'accExamCnt', 'accDefRate'], axis = 1, inplace = True)

In [67]:
data.columns = ['등록일시', '사망자', '감염자']

In [68]:
data

,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


다시 sql

In [75]:
sql = """
    select
    `Country` as '국가',
    `Units Sold` as '판매개수',
    `Unit Price` as '가격',
    `Unit Cost` as '원가',
    (`Units Sold` * `Unit Price`) as '총수익',
    (`Units Sold` * `Unit Cost`) as '총비용',
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from
    `Sales Records`
"""

test.sql_query(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20
